# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install -r /content/drive/MyDrive/Outskill/rag_day7/assignments/requirements.txt

In [15]:
import os
from getpass import getpass
from google.colab import userdata

os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

In [16]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

Create a simple RAG backend that can initialize the database and answer queries.


In [17]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""

    def __init__(self):
        self.index = None
        self.setup_settings()

    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        api_key = os.getenv("OPENROUTER_API_KEY")
        if api_key:
            Settings.llm = OpenRouter(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )

        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )

        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50

    def initialize_database(self, data_folder="/content/drive/MyDrive/Outskill/rag_day6/session_2/data"):
        """Initialize the vector database with documents."""
        # Check if data folder exists
        if not Path(data_folder).exists():
            return f"❌ Data folder '{data_folder}' not found!"

        try:
            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="./basic_rag_vectordb",
                table_name="documents"
            )

            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()

            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                show_progress=True
            )

            return f"✅ Database initialized successfully with {len(documents)} documents!"

        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"

    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first!"

        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"

        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)

        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("🚀 RAG Backend initialized and ready!")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


🚀 RAG Backend initialized and ready!


## 🎨 Part 3: Gradio Interface

Create a simple Gradio interface with:
1. Button to initialize the database
2. Text input for queries
3. Button to submit queries
4. Text output for responses
5. Text output for status messages


In [23]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""

    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()

    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)

    # Create Gradio interface using gr.Blocks() with custom theme
    with gr.Blocks(
        title="Basic RAG Assistant",
        theme=gr.themes.Soft(
            primary_hue="blue",
            secondary_hue="cyan",
            neutral_hue="slate"
        ),
        css="""
        .gradio-container {
            max-width: 900px !important;
        }
        .header-text {
            text-align: center;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            font-weight: bold;
        }
        .section-header {
            background: linear-gradient(to right, #f3f4f6, #e5e7eb);
            padding: 12px;
            border-radius: 8px;
            margin: 10px 0;
            color: #1f2937 !important;
        }
        .section-header h3 {
            color: #1f2937 !important;
            margin: 0 !important;
            font-weight: 600 !important;
        }
        .info-box {
            background: #eff6ff;
            border-left: 4px solid #3b82f6;
            padding: 12px;
            border-radius: 4px;
            margin: 10px 0;
            color: #1e40af !important;
        }
        .info-box b {
            color: #1e3a8a !important;
        }
        """
    ) as interface:

        # Title and description
        gr.Markdown(
            """
            # 🤖 RAG Assistant
            ### Intelligent Document Question-Answering System

            <div class="info-box">
            📚 <b>How it works:</b> This system uses Retrieval-Augmented Generation (RAG) to answer your questions based on the documents in your data folder.
            </div>
            """
        )

        # Initialization Section
        gr.Markdown(
            """
            <div class="section-header">
            <h3>📁 Step 1: Initialize Vector Database</h3>
            </div>
            """
        )

        gr.Markdown(
            """
            Click the button below to load and index your documents. This process may take a few moments depending on the number of documents.
            """
        )

        with gr.Row():
            init_btn = gr.Button(
                "🚀 Initialize Database",
                variant="primary",
                size="lg",
                scale=2
            )

        status_output = gr.Textbox(
            label="Initialization Status",
            placeholder="Click 'Initialize Database' to start...",
            lines=2,
            max_lines=4,
            interactive=False,
            show_label=True
        )

        gr.Markdown("---")

        # Query Section
        gr.Markdown(
            """
            <div class="section-header">
            <h3>💬 Step 2: Ask Your Questions</h3>
            </div>
            """
        )

        gr.Markdown(
            """
            Enter your question below and get AI-powered answers based on your documents.
            """
        )

        query_input = gr.Textbox(
            label="Your Question",
            placeholder="e.g., What are the main topics discussed in the documents?",
            lines=2,
            max_lines=5,
            show_label=True,
            interactive=True
        )

        with gr.Row():
            submit_btn = gr.Button(
                "🔍 Ask Question",
                variant="primary",
                size="lg",
                scale=2
            )
            clear_btn = gr.Button(
                "🗑️ Clear",
                variant="secondary",
                size="lg",
                scale=1
            )

        response_output = gr.Textbox(
            label="AI Response",
            placeholder="Your answer will appear here...",
            lines=10,
            max_lines=20,
            interactive=False,
            show_label=True,
            show_copy_button=True
        )

        # Example questions
        gr.Markdown(
            """
            ### 💡 Example Questions to Try:
            """
        )

        gr.Examples(
            examples=[
                ["What are the main topics in the documents?"],
                ["Summarize the key findings"],
                ["Explain the methodology used"],
                ["What are the most important conclusions?"],
                ["List the key recommendations"]
            ],
            inputs=query_input,
            label="Click on any example to try it"
        )

        # Footer
        gr.Markdown(
            """
            ---
            <div style="text-align: center; color: #6b7280; font-size: 0.875rem;">
            🔒 <i>Your data is processed locally and securely</i>
            </div>
            """
        )

        # Connect buttons to functions
        init_btn.click(
            fn=initialize_db,
            inputs=None,
            outputs=[status_output]
        )

        submit_btn.click(
            fn=handle_query,
            inputs=[query_input],
            outputs=[response_output]
        )

        # Clear button functionality
        clear_btn.click(
            fn=lambda: ("", ""),
            inputs=None,
            outputs=[query_input, response_output]
        )

        # Allow Enter key to submit
        query_input.submit(
            fn=handle_query,
            inputs=[query_input],
            outputs=[response_output]
        )

    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")

/tmp/ipython-input-177562971.py:13: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-177562971.py:13: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(


✅ Basic RAG interface created successfully!


## 🚀 Part 4: Launch Your Application

Launch your Gradio application and test it!


In [24]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")
print("")
print("🚀 Launch your app:")

# Your launch code here:
# Uncomment when implemented
basic_interface.launch()

🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used

🚀 Launch your app:
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9bf9a34f14f2c55fc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

- [x] RAG backend is provided and working
- [ ] Created Gradio interface with required components:
  - [ ] Title and description using gr.Markdown()
  - [ ] Initialize database button using gr.Button()
  - [ ] Status output using gr.Textbox()
  - [ ] Query input field using gr.Textbox()
  - [ ] Submit query button using gr.Button()
  - [ ] Response output area using gr.Textbox()
- [ ] Connected buttons to backend functions using .click()
- [ ] Successfully launched the application
- [ ] Tested the full workflow (initialize → query → response)

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

- A functional web interface for your RAG system
- Understanding of Gradio basics and component connections
- A foundation for building more complex AI applications

**Next Steps**: Complete Assignment 3b to add advanced configuration options to your RAG interface!
